# Scrapping on WoS to get dismissed and undismissed authors

In [1]:
import time
from scraper_WoS import WosClient
import pandas as pd
from unicode_tr.extras import slugify

In [2]:
def get_author_names(path_text_file):
#getting all the names 
    searchnames = []
    i=0
    for line in open(path_text_file,'r'):
        QueryString = line.strip()
        QueryString = "AU="+line.split(',')[0].split()[-1]+' '+line.split(',')[0].split()[0]
        searchnames.append(QueryString)
    return  searchnames, QueryString

In [3]:
def searching_co_authors_WoS(searchnames):
    soap = WosClient(lite=True)
    soap.connect()
    names = []
    lens = []
    co_aut = []
    i=0
    for i in range(len(searchnames)):
        #line = SearchNames.SearchNames[i]
        QueryString = searchnames[i]
        #QueryString = line
        print(QueryString)
        if i == 2400: # WOS constraints that each ID can only search for 2500 times. Just set smaller counts to renew the ID.
            soap = WosClient(lite=True)
            soap.connect()
        results = soap.search(QueryString)
        #getting number of name of co-authors
        if (results.recordsFound>0):
            authors = []
            for j in range (len(results.records)):
                for k in range (len(results.records[j].authors)):
                    for l in range (len(results.records[j].authors[k].value)):
                        co_authors=results.records[j].authors[k].value[l]
                        authors.append(co_authors)
                        print('co_author',co_authors)
        print(results.recordsFound)
        names.append(QueryString.split('=')[1])
        lens.append(results.recordsFound)
        co_aut.append(authors)
        time.sleep(0.5) # WOS constraints that you can only search twice per second
        i=i+1        
    return names, lens, co_aut

In [4]:
#a python module to make unicode strings work as expected for turkish chars. solves the turkish "İ" problem

# have to do pip install unicode_tr
def translate_names(names):
    names_en = []
    for i in range(len(names)):
        name_en = slugify(names[i])
        name_en = name_en.replace('-',' ').title()
        names_en.append(name_en)
    return names_en


In [ ]:
#formatting co authors column to match with authors
#for i in range(df.shape[0]):
#    for j in range(len(df.CoAuthors.iloc[i])):
#        a=df.CoAuthors.iloc[i]
#        a[j]=a[j].replace(","," ")
#        df.CoAuthors.iloc[i] = a
#df

In [5]:
def count_nb_coauthors(co_aut):
    cnt_coauthors=0
    for l in co_aut:
        cnt_coauthors += len(l)
    print('Totally ' + str(cnt_coauthors) + ' guys were downloaded as coauthors')
    return cnt_coauthors

In [6]:
def save_csv(list1,list2):
    dismissed_df = pd.DataFrame()
    dismissed_df['AU']=list(list1)
    dismissed_df.to_csv('../data/dismissed.csv')
    undismissed_df = pd.DataFrame()
    undismissed_df['AU']=list(list2)
    undismissed_df.to_csv('../data/undismissed.csv')


# Extract authors

In [7]:
searchnames, QueryString = get_author_names('../Data/Dismiss_Acad_List.txt')
names,lens,co_aut = searching_co_authors_WoS(searchnames)
names_en=translate_names(names)

#dataset with dimissed people with their correspondant coauthors and number of papers
df = pd.DataFrame([names,lens,co_aut]).T
df.columns=['Author','NoPapers','CoAuthors']

nb_coauthors = count_nb_coauthors(co_aut)

coauthors = set()
for l in co_aut:
    for author in l:
        author = author.replace(',', '')
        coauthors.add(author)

dismissed = set(names_en)
print('Totally we have ' + str(len(dismissed)) + ' dismissed and ' + str(len(coauthors)) + ' unique coauthors')

undismissed = coauthors - dismissed
print('So, we have ' + str(len(undismissed)) + ' undismissed guys')

save_csv(dismissed,undismissed)

Authenticated (SID: D4uOQ5te63uKeuvin6G)
AU=AYDIN AYDIN
co_author Kul, Sibel
co_author Aydin, Aydin
co_author Dinc, Hasan
co_author Erduran, Erol
co_author Aydin, Leyla
co_author Aydin, Aydin
co_author Bulbul, Baris
co_author Ozkalkan, Beliz
co_author Ozaydin, Cansu
co_author Inaltekin, Ezgi
co_author Aydin, Aydin
co_author Aksoy, Hamit Zafer
co_author Reis, Abdulkadir
co_author Ozdemir, Feyyaz
co_author Zengin, Ahmet Y.
co_author Topkan, Erkan
co_author Aydin, Aydin
co_author Reis, A. Kadir
4
AU=GİLGİL ERDAL
co_author Melikoglu, Meltem Alkan
co_author Kocabas, Hilal
co_author Sezer, Ilhan
co_author Akdag, Ali
co_author Gilgil, Erdal
co_author Butun, Bulent
co_author Dundar, Umit
co_author Kupesiz, Alphan
co_author Ozdem, Sehahat
co_author Gilgil, Erdal
co_author Tuncer, Tiraje
co_author Yesilipek, Akif
co_author Gultekin, Meral
co_author Gilgil, Erdal
co_author Kacar, Cahit
co_author Tuncer, Tiraje
co_author Tuncer, Tiraje
co_author Gilgil, Erdal
co_author Kacar, Cahit
co_author Kurta

AU=ŞENGÜL ABDULLAH
co_author Emsley, Robin
co_author Hargarter, Ludger
co_author Bergmans, Paul
co_author Uglesic, Boran
co_author Sengul, Abdullah Cem
co_author Petralia, Antonino
co_author Khannanova, Angelina
co_author Cherubin, Pierre
co_author Schreiner, Andreas
co_author Karaoglan, Mustafa
co_author Tutar, Suleyman
co_author Sengul, Abdullah Murat
co_author Tekeli, Hakan
co_author Togrol, Rifat Erdem
co_author Sengul, Abdullah
3


KeyboardInterrupt: 

In [72]:
len(undismissed)


25047